<a href="https://colab.research.google.com/github/chaiai/deep-learning/blob/main/CIFAR_10_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 2: CIFAR-10 autoencoder

For this week, you will create a convolutional autoencoder for the CIFAR10 dataset. You are free to choose the architecture of your autoencoder provided that the output image has the same dimensions as the input image.

After training, your model should meet loss and accuracy requirements when evaluated with the test dataset. You will then download the model and upload it in the classroom for grading.

Let's begin!

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import Sequential

In [ ]:
# preprocessing function
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0

  return image, image # dataset label is not used. replaced with the same image input.

# parameters
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024

# use tfds.load() to fetch the 'train' split of CIFAR-10
train_dataset = tfds.load('cifar10', as_supervised = True, split = 'train')

# preprocess the dataset with the `map_image()` function above
train_dataset = train_dataset.map(map_image) 

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)


# use tfds.load() to fetch the 'test' split of CIFAR-10
test_dataset = tfds.load('cifar10', as_supervised = True, split = 'test')

# preprocess the dataset with the `map_image()` function above
test_dataset = test_dataset.map(map_image) 

# batch the dataset
test_dataset = test_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteZSU91C/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteZSU91C/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [ ]:
# suggested layers to use. feel free to add or remove as you see fit.
from keras.layers import Conv2D, UpSampling2D

# use the Sequential API (you can remove if you want to use the Functional API)
model = Sequential()

### START CODE HERE ###
# use `model.add()` to add layers (if using the Sequential API)
model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(tf.keras.layers.BatchNormalization())     
model.add(Conv2D(32, kernel_size=3, strides=2, padding='same', activation='relu'))      
model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu'))      
model.add(tf.keras.layers.BatchNormalization())    
model.add(UpSampling2D())
model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu'))      
model.add(tf.keras.layers.BatchNormalization()) 
model.add(Conv2D(3,  kernel_size=1, strides=1, padding='same', activation='sigmoid'))  

model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')
### END CODE HERE ###

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9

In [ ]:
# Please do not change the model.compile() parameters
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

In [ ]:
# parameters
train_steps = len(train_dataset) // BATCH_SIZE 
val_steps = len(test_dataset) // BATCH_SIZE

### START CODE HERE ###
model.fit(train_dataset, validation_data= test_dataset, epochs= 10)
### END CODE HERE ###

Epoch 1/10
391/391 [==============================] - 47s 34ms/step - loss: 0.0094 - accuracy: 0.6686 - val_loss: 0.0066 - val_accuracy: 0.7347
Epoch 2/10
391/391 [==============================] - 8s 20ms/step - loss: 0.0013 - accuracy: 0.8085 - val_loss: 9.1448e-04 - val_accuracy: 0.8329
Epoch 3/10
391/391 [==============================] - 8s 19ms/step - loss: 8.7123e-04 - accuracy: 0.8338 - val_loss: 5.9073e-04 - val_accuracy: 0.8494
Epoch 4/10
391/391 [==============================] - 8s 20ms/step - loss: 7.6266e-04 - accuracy: 0.8490 - val_loss: 5.6182e-04 - val_accuracy: 0.8194
Epoch 5/10
391/391 [==============================] - 8s 19ms/step - loss: 6.3228e-04 - accuracy: 0.8570 - val_loss: 5.9860e-04 - val_accuracy: 0.8297
Epoch 6/10
391/391 [==============================] - 8s 20ms/step - loss: 5.5690e-04 - accuracy: 0.8624 - val_loss: 5.9794e-04 - val_accuracy: 0.8621
Epoch 7/10
391/391 [==============================] - 8s 20ms/step - loss: 5.4331e-04 - accuracy: 0.8642 

In [ ]:
result = model.evaluate(test_dataset, steps=10)

10/10 [==============================] - 0s 10ms/step - loss: 3.3483e-04 - accuracy: 0.8694


In [ ]:
model.save('mymodel.h5')